In [ ]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
import torch
import torch.nn.functional as F
import csv

# Load pretrained model
model_name = "roberta-base"
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForMaskedLM.from_pretrained(model_name)
model.eval()

# ---- Load CSV once, preserving rows ----
with open(r"NGSL_lists\NEWEST_NGSL_2.csv", newline="", encoding="utf-8") as f:
    reader2 = csv.reader(f)
    csv_rows = [row for row in reader2]   # list of lists, each row preserved


def masked_tokens_probabilities(sentence: str, M, word_prob, i):
    # Tokenize once
    inputs = tokenizer(sentence, return_tensors="pt")

    # Find all mask positions
    mask_positions = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=False)

    # Forward pass once
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=-1)


    # Iterate over mask positions
    for pos in mask_positions[:, 1]:
        if pos > 2 and pos < (mask_positions[:, 1][-1] - 1):
            for j, row in enumerate(csv_rows):
                for tok in row:
                    token_id = tokenizer.convert_tokens_to_ids(tok)
                    prob = probs[0, pos, token_id].item()
                    M[i][j] += word_prob * prob


C:\Users\HP\bert_experiments\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from wordfreq import word_frequency

# Function to compute frequency of a word
def get_word_frequency(word: str, lang: str = "en") -> float:
    """
    Returns the frequency of a word in the given language.
    By default, English ("en") is used.
    
    The result is a probability between 0 and 1.
    """
    return word_frequency(word, lang)


In [ ]:
import numpy as np
import csv

with open(r"NGSL_lists\NEWEST_NGSL.csv", newline="") as f:
    n = sum(1 for _ in f)

M = np.zeros((n, n), dtype=np.float64)

i=-1

with open(r"NGSL_lists\NEWEST_NGSL.csv", newline="", encoding='utf-8') as f:
    reader = csv.reader(f)
    for row in reader:
        i+=1
        if(i%50 == 0):
            print(i)
        for word in row:
            if(word != ""):
                word_prob = get_word_frequency(word, "en")
                masked_tokens_probabilities("<mask> <mask> <mask> " + word + " <mask> <mask> <mask> <mask> <mask> <mask>", M, word_prob, i)
            
            



In [ ]:
np.save("Probabilities_NGSL.npy", M)

In [2]:
M[4494][4495]

NameError: name 'M' is not defined

In [3]:
get_word_frequency("unlimited")

NameError: name 'get_word_frequency' is not defined